In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
start_date = "2019-04-01"
end_date = "2019-05-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  comments = pd.concat(comments)


/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  issues = pd.concat(issues)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  prs = pd.concat(prs)


In [6]:
# Only keep the dates we want
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')
issues = issues.query('updatedAt > @start_date and updatedAt < @end_date')
prs = prs.query('updatedAt > @start_date and updatedAt < @end_date')

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/nteract](https://github.com/nteract)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-04-01** to **2019-05-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 30 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                      repo  author
  46          nteract                 papermill       6
  45          nteract                  hydrogen       2
  49          nteract                      vdom       2
  47          nteract                 scrapbook       1
  48          nteract                  semiotic       1
  38       jupyterlab                jupyterlab     291
  41       jupyterlab            jupyterlab-git      56
  39       jupyterlab       jupyterlab-celltags      15
  40       jupyterlab  jupyterlab-data-explorer       5
  42       jupyterlab         jupyterlab-github       5
  43       jupyterlab   jupyterlab-google-drive       5
  44       jupyterlab              team-compass       5
  31       jupyterhub       mybinder.org-deploy      66
  37       jupyterhub    zero-to-jupyterhub-k8s      38
  28       jupyterhub                jupyterhub      30
  24       jupyterhub                 binderhub      24
  30       jupyterhub               kubespawner      12
  33       jupyterhub            systemdspawner      10
  35       jupyterhub   the-littlest-jupyterhub      10
  22       jupyterhub            binder-billing       6
  26       jupyterhub             dockerspawner       6
  29       jupyterhub     kerberosauthenticator       6
  34       jupyterhub              team-compass       6
  20       jupyterhub      alabaster-jupyterhub       4
  27       jupyterhub      jupyter-server-proxy       4
  36       jupyterhub               yarnspawner       4
  21       jupyterhub                    binder       2
  23       jupyterhub               binder-data       2
  25       jupyterhub   configurable-http-proxy       2
  32       jupyterhub       nativeauthenticator       2
  17  jupyter-widgets                ipyleaflet      36
  19  jupyter-widgets                 pythreejs      16
  18  jupyter-widgets                ipywidgets      12
  11          jupyter                 nbconvert      19
  16          jupyter               repo2docker      13
  3           jupyter             docker-stacks      12
  7           jupyter         jupyter.github.io       7
  5           jupyter                   jupyter       6
  14          jupyter                  notebook       6
  4           jupyter        enterprise_gateway       5
  6           jupyter              jupyter-book       5
  8           jupyter            jupyter_client       3
  10          jupyter                     nb2kg       2
  13          jupyter                  nbviewer       2
  15          jupyter                 qtconsole       2
  9           jupyter            kernel_gateway       1
  12          jupyter                  nbformat       1
  0           ipython                   ipython      15
  1           ipython          ipython-in-depth       9
  2           ipython                 traitlets       9,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['papermill', 'hydrogen', 'vdom', 'scrapbook', 'semiotic', 'jupyterlab', 'jupyterlab-git', 'jupyterlab-celltags', 'jupyterlab-data-explorer', 'jupyterlab-github', 'jupyterlab-google-drive', 'team-compass', 'mybinder.org-deploy', 'zero-to-jupyterhub-k8s', 'jupyterhub', 'binderhub', 'kubespawner', 'systemdspawner', 'the-littlest-jupyterhub', 'binder-billing', 'dockerspawner', 'kerberosauthenticator', 'team-compass', 'alabaster-jupyterhub', 'jupyter-server-proxy', 'yarnspawner', 'binder', 'binder-data', 'configurable-http-proxy', 'nativeauthenticator', 'ipyleaflet', 'pythreejs', 'ipywidgets', 'nbconvert', 'repo2docker', 'docker-stacks', 'jupyter.github.io', 'jupyter', 'notebook', 'enterprise_gateway', 'jupyter-book', 'jupyter_client', 'nb2kg', 'nbviewer', 'qtconsole', 'kernel_gateway', 'nbformat', 'ipython', 'ipython-in-depth', 'traitlets'],
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merged

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:        username  authored authorAssociation
    5     switowski         9       CONTRIBUTOR
    2    dan-coates         6       CONTRIBUTOR
    0  NumesSanguis         3              NONE
    1       albertz         3       CONTRIBUTOR
    3      pickfire         3       CONTRIBUTOR
    4      rmorshea         3      COLLABORATOR
    6       travigd         3       CONTRIBUTOR
    7          yamt         3              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:            username  authored authorAssociation
    30          parente        14            MEMBER
    2             MSeal         8            MEMBER
    7           betatim         8            MEMBER
    9         choldgraf         8            MEMBER
    31         t-makaro         6      COLLABORATOR
    27            minrk         3            MEMBER
    35           witten         2       CONTRIBUTOR
    6        alexmorley         2       CONTRIBUTOR
    22          jhamman         2       CONTRIBUTOR
    32        takluyver         2            MEMBER
    0           Carreau         1            MEMBER
    28           mpacer         1            MEMBER
    24           kojix2         1       CONTRIBUTOR
    25  maartenbreddels         1       CONTRIBUTOR
    26        mattjtodd         1       CONTRIBUTOR
    33        vilhelmen         1       CONTRIBUTOR
    29      nils-werner         1       CONTRIBUTOR
    34         willingc         1            MEMBER
    36           xmnlab         1       CONTRIBUTOR
    37           ying-w         1              NONE
    23      kevin-bates         1            MEMBER
    19          hroncok         1       CONTRIBUTOR
    21    ivigamberdiev         1       CONTRIBUTOR
    12          djhoese         1       CONTRIBUTOR
    3            Szauka         1       CONTRIBUTOR
    4         akchinSTC         1      COLLABORATOR
    5      alasdairtran         1       CONTRIBUTOR
    8      captainsafia         1            MEMBER
    10           chuyqa         1       CONTRIBUTOR
    11            clkao         1       CONTRIBUTOR
    13          dsblank         1            MEMBER
    20         impact27         1       CONTRIBUTOR
    14         emiliotl         1       CONTRIBUTOR
    15         ericdill         1       CONTRIBUTOR
    16           esevan         1       CONTRIBUTOR
    17     flying-sheep         1       CONTRIBUTOR
    18  henriqueribeiro         1       CONTRIBUTOR
    1   GrahamDumpleton         1       CONTRIBUTOR
    38           ykgmfq         1       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:        org              repo   username  closed
    0  ipython           ipython    Carreau      15
    1  ipython  ipython-in-depth    Carreau       6
    2  ipython  ipython-in-depth  blink1073       3
    3  ipython         traitlets      minrk       9,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:         org                repo     username  closed
    4   jupyter       docker-stacks        minrk       1
    5   jupyter       docker-stacks      parente      11
    6   jupyter  enterprise_gateway  kevin-bates       4
    7   jupyter  enterprise_gateway     lresende       1
    8   jupyter             jupyter      Carreau       1
    9   jupyter             jupyter    choldgraf       2
    10  jupyter             jupyter      parente       3
    11  jupyter        jupyter-book    choldgraf       5
    12  jupyter   jupyter.github.io         Ruv7       1
    13  jupyter   jupyter.github.io        minrk       1
    14  jupyter   jupyter.github.io      parente       3
    15  jupyter   jupyter.github.io     willingc       2
    16  jupyter      jupyter_client      Carreau       1
    17  jupyter      jupyter_client        minrk       1
    18  jupyter      jupyter_client    takluyver       1
    19  jupyter      kernel_gateway     rolweber       1
    20  jupyter               nb2kg  kevin-bates       2
    21  jupyter           nbconvert        MSeal      12
    22  jupyter           nbconvert       mpacer       6
    23  jupyter           nbconvert    takluyver       1
    24  jupyter            nbformat        MSeal       1
    25  jupyter            nbviewer      parente       2
    26  jupyter            notebook        minrk       1
    27  jupyter            notebook    takluyver       5
    28  jupyter           qtconsole   ccordoba12       2
    29  jupyter         repo2docker      betatim       7
    30  jupyter         repo2docker    choldgraf       1
    31  jupyter         repo2docker       lheagy       1
    32  jupyter         repo2docker        minrk       2
    33  jupyter         repo2docker    yuvipanda       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 30 days',
    width: 1000
  }), Chart({
    data:                 org        repo       username  closed
    34  jupyter-widgets  ipyleaflet    martinRenou      36
    35  jupyter-widgets  ipywidgets  SylvainCorlay      12
    36  jupyter-widgets   pythreejs        vidartf      16,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter-widgets in the last 30 days',
    width: 1000
  }), Chart({
    data:            org                     repo      username  closed
    37  jupyterhub     alabaster-jupyterhub       betatim       2
    38  jupyterhub     alabaster-jupyterhub     choldgraf       2
    39  jupyterhub                   binder       betatim       2
    40  jupyterhub           binder-billing     choldgraf       6
    41  jupyterhub              binder-data     choldgraf       2
    42  jupyterhub                binderhub       betatim      14
    43  jupyterhub                binderhub         minrk       6
    44  jupyterhub                binderhub     sgibson91       2
    45  jupyterhub                binderhub     yuvipanda       2
    46  jupyterhub  configurable-http-proxy         minrk       2
    47  jupyterhub            dockerspawner         minrk       6
    4

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                     repo  number  \
  0           ipython                ipykernel      12   
  1           ipython                  ipython      31   
  2           ipython                traitlets       6   
  3           jupyter            accessibility       1   
  4           jupyter            atom-notebook       1   
  5           jupyter                   design       1   
  6           jupyter            docker-stacks       3   
  7           jupyter       enterprise_gateway       2   
  8           jupyter                  jupyter       4   
  9           jupyter             jupyter-book       1   
  10          jupyter           jupyter_client       2   
  11          jupyter          jupyter_console       2   
  12          jupyter     kernel_gateway_demos       1   
  13          jupyter                nbconvert       1   
  14          jupyter                   nbdime       1   
  15          jupyter                 notebook      34   
  16  jupyter-widgets               ipyleaflet       5   
  17  jupyter-widgets               ipywidgets      56   
  18  jupyter-widgets       jupyterlab-sidecar       8   
  19  jupyter-widgets                pythreejs      16   
  20  jupyter-widgets               traittypes       4   
  21  jupyter-widgets      widget-cookiecutter       8   
  22  jupyter-widgets   widget-ts-cookiecutter       4   
  23       jupyterhub     alabaster-jupyterhub       4   
  24       jupyterhub           binder-billing       2   
  25       jupyterhub                binderhub       8   
  26       jupyterhub            dockerspawner       2   
  27       jupyterhub     jupyter-server-proxy       2   
  28       jupyterhub               jupyterhub       5   
  29       jupyterhub     jupyterhub-on-hadoop       4   
  30       jupyterhub           jupyterlab-hub       4   
  31       jupyterhub              kubespawner      14   
  32       jupyterhub        ldapauthenticator       2   
  33       jupyterhub           mybinder-tools       2   
  34       jupyterhub      mybinder.org-deploy       6   
  35       jupyterhub      nativeauthenticator       3   
  36       jupyterhub           oauthenticator       4   
  37       jupyterhub              sudospawner       2   
  38       jupyterhub             team-compass       2   
  39       jupyterhub   zero-to-jupyterhub-k8s      40   
  40       jupyterlab        jupyter-renderers      10   
  41       jupyterlab               jupyterlab     105   
  42       jupyterlab    jupyterlab-commenting       5   
  43       jupyterlab           jupyterlab-git      20   
  44       jupyterlab  jupyterlab-google-drive      15   
  45       jupyterlab         jupyterlab-latex      10   
  46       jupyterlab        jupyterlab-monaco       5   
  47       jupyterlab           jupyterlab-toc      10   
  48       jupyterlab             team-compass       5   
  49          nteract                bookstore       2   
  50          nteract                 commuter       2   
  51          nteract                       dx       1   
  52          nteract                  nteract      14   
  53          nteract                papermill       1   
  54          nteract                scrapbook       3   
  55          nteract                 semiotic       1   
  56          nteract            semiotic-docs       3   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                          ipython/ipython  
  2                        ipython/traitlets  
  3                    jupyter/accessibility  
  4                    jupyter/atom-notebook  
  5                           jupyter/design  
  6                    jupyter/docker-stacks  
  7               jupyter/enterprise_gateway  
  8                          jupyter/jupyter  
  9                     jupyter/jupyter-book  
  10                  jupyter/jupyter_client  
  11                 jupyter/jupyter_console  
  12    

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                     repo  number  \
  0   binder-examples                        r       4   
  1   binder-examples             requirements       2   
  2           ipython                  ipython      19   
  3           ipython                traitlets      39   
  4           jupyter            docker-stacks       6   
  5           jupyter       enterprise_gateway      10   
  6           jupyter                  jupyter       2   
  7           jupyter             jupyter-book       1   
  8           jupyter           jupyter-sphinx       2   
  9           jupyter           jupyter_client       1   
  10          jupyter                    nb2kg       3   
  11          jupyter                nbconvert       9   
  12          jupyter                   nbdime       1   
  13          jupyter                 nbviewer       2   
  14          jupyter                 notebook      16   
  15          jupyter                qtconsole       2   
  16          jupyter              repo2docker       6   
  17  jupyter-widgets               ipyleaflet      28   
  18  jupyter-widgets               ipywidgets      77   
  19  jupyter-widgets                pythreejs      24   
  20  jupyter-widgets   widget-ts-cookiecutter       4   
  21       jupyterhub                   binder       2   
  22       jupyterhub                binderhub       2   
  23       jupyterhub            dockerspawner       8   
  24       jupyterhub     jupyter-server-proxy       8   
  25       jupyterhub               jupyterhub      32   
  26       jupyterhub     jupyterhub-on-hadoop      10   
  27       jupyterhub              kubespawner       8   
  28       jupyterhub        ldapauthenticator       2   
  29       jupyterhub      mybinder.org-deploy       2   
  30       jupyterhub           oauthenticator       2   
  31       jupyterhub           systemdspawner       2   
  32       jupyterhub             team-compass      14   
  33       jupyterhub  the-littlest-jupyterhub      10   
  34       jupyterhub            traefik-proxy       2   
  35       jupyterhub              yarnspawner       6   
  36       jupyterhub   zero-to-jupyterhub-k8s      22   
  37       jupyterlab               jupyterlab     270   
  38       jupyterlab      jupyterlab-celltags      10   
  39       jupyterlab           jupyterlab-git      75   
  40       jupyterlab        jupyterlab-github       5   
  41       jupyterlab  jupyterlab-google-drive      10   
  42       jupyterlab         jupyterlab-latex       5   
  43       jupyterlab             team-compass       5   
  44          nteract                bookstore       1   
  45          nteract                 commuter       6   
  46          nteract                 hydrogen      12   
  47          nteract           minimal-plotly       1   
  48          nteract                papermill      11   
  49          nteract                scrapbook       1   
  50          nteract                 semiotic       7   
  51          nteract                     vdom       4   
  
                                    org/repo  
  0                        binder-examples/r  
  1             binder-examples/requirements  
  2                          ipython/ipython  
  3                        ipython/traitlets  
  4                    jupyter/docker-stacks  
  5               jupyter/enterprise_gateway  
  6                          jupyter/jupyter  
  7                     jupyter/jupyter-book  
  8                   jupyter/jupyter-sphinx  
  9                   jupyter/jupyter_client  
  10                           jupyter/nb2kg  
  11                       jupyter/nbconvert  
  12                          jupyter/nbdime  
  13                        jupyter/nbviewer  
  14                        jupyter/notebook  
  15                       jupyter/qtconsole  
  16                     jupyter/repo2docker  
  17              jupyter-widgets/ipyleaflet  
  18            

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             org                     repo     kind  count
    40   jupyterlab        jupyter-renderers  created   10.0
    41   jupyterlab               jupyterlab  created  105.0
    42   jupyterlab    jupyterlab-commenting  created    5.0
    43   jupyterlab           jupyterlab-git  created   20.0
    44   jupyterlab  jupyterlab-google-drive  created   15.0
    45   jupyterlab         jupyterlab-latex  created   10.0
    47   jupyterlab           jupyterlab-toc  created   10.0
    48   jupyterlab             team-compass  created    5.0
    69   jupyterlab      jupyterlab-celltags  created    0.0
    70   jupyterlab        jupyterlab-github  created    0.0
    114  jupyterlab        jupyter-renderers   closed    0.0
    115  jupyterlab               jupyterlab   closed  270.0
    116  jupyterlab    jupyterlab-commenting   closed    0.0
    117  jupyterlab           jupyterlab-git   closed   75.0
    118  jupyterlab  jupyterlab-google-drive   closed   10.0
    119  jupyterlab         jupyterlab-latex   closed    5.0
    121  jupyterlab           jupyterlab-toc   closed    0.0
    122  jupyterlab             team-compass   closed    5.0
    143  jupyterlab      jupyterlab-celltags   closed   10.0
    144  jupyterlab        jupyterlab-github   closed    5.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                 org                    repo     kind  count
    16  jupyter-widgets              ipyleaflet  created    5.0
    17  jupyter-widgets              ipywidgets  created   56.0
    18  jupyter-widgets      jupyterlab-sidecar  created    8.0
    19  jupyter-widgets               pythreejs  created   16.0
    20  jupyter-widgets              traittypes  created    4.0
    21  jupyter-widgets     widget-cookiecutter  created    8.0
    22  jupyter-widgets  widget-ts-cookiecutter  created    4.0
    90  jupyter-widgets              ipyleaflet   closed   28.0
    91  jupyter-widgets              ipywidgets   closed   77.0
    92  jupyter-widgets      jupyterlab-sidecar   closed    0.0
    93  jupyter-widgets               pythreejs   closed   24.0
    94  jupyter-widgets              traittypes   closed    0.0
    95  jupyter-widgets     widget-cookiecutter   closed    0.0
    96  jupyter-widgets  widget-ts-cookiecutter   closed    4.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for jupyter-widgets',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:         org       repo     kind  count
    0   ipython  ipykernel  created   12.0
    1   ipython    ipython  created   31.0
    2   ipython  traitlets  created    6.0
    74  ipython  ipykernel   closed    0.0
    75  ipython    ipython   closed   19.0
   

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                     repo   time_open  \
  0   binder-examples                        r  158.033397   
  1   binder-examples             requirements    0.878611   
  2           ipython                  ipython    5.773090   
  3           ipython                traitlets  524.997407   
  4           jupyter            docker-stacks  135.174954   
  5           jupyter       enterprise_gateway    2.735359   
  6           jupyter                  jupyter  528.921007   
  7           jupyter             jupyter-book    1.337373   
  8           jupyter           jupyter-sphinx  347.251921   
  9           jupyter           jupyter_client   56.048183   
  10          jupyter                    nb2kg    7.627292   
  11          jupyter                nbconvert   47.337720   
  12          jupyter                   nbdime   61.557141   
  13          jupyter                 nbviewer   28.592737   
  14          jupyter                 notebook    7.112940   
  15          jupyter                qtconsole   21.478519   
  16          jupyter              repo2docker   56.813437   
  17  jupyter-widgets               ipyleaflet    1.409132   
  18  jupyter-widgets               ipywidgets    2.938796   
  19  jupyter-widgets                pythreejs   19.376806   
  20  jupyter-widgets   widget-ts-cookiecutter    1.007500   
  21       jupyterhub                   binder    3.087928   
  22       jupyterhub                binderhub   90.219317   
  23       jupyterhub            dockerspawner   26.389468   
  24       jupyterhub     jupyter-server-proxy    1.095995   
  25       jupyterhub               jupyterhub    1.573709   
  26       jupyterhub     jupyterhub-on-hadoop    5.924155   
  27       jupyterhub              kubespawner   76.931319   
  28       jupyterhub        ldapauthenticator    0.075509   
  29       jupyterhub      mybinder.org-deploy   33.330590   
  30       jupyterhub           oauthenticator   47.958796   
  31       jupyterhub           systemdspawner   21.074907   
  32       jupyterhub             team-compass   25.022396   
  33       jupyterhub  the-littlest-jupyterhub   14.536076   
  34       jupyterhub            traefik-proxy    1.121551   
  35       jupyterhub              yarnspawner    0.991076   
  36       jupyterhub   zero-to-jupyterhub-k8s    3.088750   
  37       jupyterlab               jupyterlab   32.962014   
  38       jupyterlab      jupyterlab-celltags  182.726638   
  39       jupyterlab           jupyterlab-git   12.142292   
  40       jupyterlab        jupyterlab-github   52.334549   
  41       jupyterlab  jupyterlab-google-drive   49.187031   
  42       jupyterlab         jupyterlab-latex    0.994745   
  43       jupyterlab             team-compass    5.495081   
  44          nteract                bookstore   21.619711   
  45          nteract                 commuter   86.714855   
  46          nteract                 hydrogen    7.129664   
  47          nteract           minimal-plotly   26.437975   
  48          nteract                papermill   20.318819   
  49          nteract                scrapbook    0.928993   
  50          nteract                 semiotic    5.082326   
  51          nteract                     vdom  106.607894   
  
                                    org/repo  
  0                        binder-examples/r  
  1             binder-examples/requirements  
  2                          ipython/ipython  
  3                        ipython/traitlets  
  4                    jupyter/docker-stacks  
  5               jupyter/enterprise_gateway  
  6                          jupyter/jupyter  
  7                     jupyter/jupyter-book  
  8                   jupyter/jupyter-sphinx  
  9                   jupyter/jupyter_client  
  10                           jupyter/nb2kg  
  11                       jupyter/nbconvert  
  12                          jupyter/nbdime  
  13                        jupyter/nbv

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 30 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             author  count authorAssociation
    4        choldgraf      8            MEMBER
    0  MarijaStanojcic      2              NONE
    1       RAJohansen      2              NONE
    2          betatim      2            MEMBER
    3         cboettig      2            MEMBER
    5         schluenz      2              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:                 author  count authorAssociation
    0              Carreau    133            MEMBER
    49            rmorshea     72      COLLABORATOR
    5       LucianaMarques     33       CONTRIBUTOR
    44               minrk     18            MEMBER
    61              wanick     15              NONE
    34             jakevdp     12              NONE
    46             panaali     12              NONE
    32             hroncok     12       CONTRIBUTOR
    64             zzzz-qq      9       CONTRIBUTOR
    39  keisuke-yanagisawa      9              NONE
    43         martinRenou      9       CONTRIBUTOR
    2       DanielGoldfarb      9              NONE
    25        davidszotten      9              NONE
    20           blackball      8              NONE
    35          jasongrout      6            MEMBER
    33           hyperknot      6              NONE
    28          dwhswenson      6              NONE
    54       thequackdaddy      6              NONE
    40                krey      6              NONE
    10                OW51      6              NONE
    13       SylvainCorlay      6            MEMBER
    14           ThetomekK      6              NONE
    57         tommydevitt      3              NONE
    48            ressafri      3              NONE
    63           yuvipanda      3      COLLABORATOR
    6                MSeal      3       CONTRIBUTOR
    45              mrrech      3              NONE
    62             weaming      3              NONE
    4            Hazarmavi      3              NONE
    47            prjemian      3              NONE
    3           DannyFeliz      3              NONE
    58            tritemio      3              NONE
    50             sanyam5      3       CONTRIBUTOR
    51        sjpfenninger      3              NONE
    52            stphnlyd      3              NONE
    60              waboke      3              NONE
    42             lvc0107      3              NONE
    59         vladserkoff      3              NONE
    55       thomasaarholt      3              NONE
    56           tirkarthi      3              NONE
    53          texadactyl      3              NONE
    8        MatthewWilkes      3              NONE
    41             leolinf      3              NONE
    23              daharn      3       CONTRIBUTOR
    11             SKPriya      3              NONE
    12          SimonBiggs      3              NONE
    15              WestXu      3              NONE
    16        aquirdTurtle      3              NONE
    17          aslily1234      3              NONE
    18          basnijholt      3              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],


## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             author  n_first_responses authorAssociation
    1        choldgraf                  2            MEMBER
    0  MarijaStanojcic                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:                 author  n_first_responses authorAssociation
    0              Carreau                 24            MEMBER
    35            rmorshea                 18      COLLABORATOR
    30               minrk                  3            MEMBER
    45              wanick                  2              NONE
    22             hroncok                  2       CONTRIBUTOR
    8                 OW51                  2              NONE
    39       thequackdaddy                  2              NONE
    27          kak-to-tak                  1              NONE
    28  keisuke-yanagisawa                  1              NONE
    29         martinRenou                  1       CONTRIBUTOR
    31              mrrech                  1              NONE
    32             panaali                  1              NONE
    33            prjemian                  1              NONE
    34            ressafri                  1              NONE
    37        sjpfenninger                  1              NONE
    36             sanyam5                  1       CONTRIBUTOR
    25          jclevesque                  1              NONE
    38          texadactyl                  1              NONE
    40       thomasaarholt                  1              NONE
    41           tirkarthi                  1              NONE
    42         tommydevitt                  1              NONE
    43            tritemio                  1              NONE
    44              waboke                  1              NONE
    46             weaming                  1              NONE
    26         jerrykaplan                  1              NONE
    24          jasongrout                  1            MEMBER
    1            DSamuylov                  1              NONE
    23           hyperknot                  1              NONE
    2       DanielGoldfarb                  1              NONE
    3           DannyFeliz                  1              NONE
    4       LucianaMarques                  1              NONE
    5       Manav-Sarvaiya                  1              NONE
    6        MatthewWilkes                  1              NONE
    7         NumesSanguis                  1              NONE
    9            ThetomekK                  1              NONE
    10              WestXu                  1              NONE
    11        aquirdTurtle                  1              NONE
    12          basnijholt                  1              NONE
    13          bergtholdt                  1       CONTRIBUTOR
    14           blink1073                  1            MEMBER
    15               cdeil                  1              NONE
    16         datdinhquoc                  1              NONE
    17        davidszotten                  1              NONE
    18           dedalus94                  1              NONE
    19       diallobakary4                  1              NONE
    20          dwhswenson  

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>